In [24]:
#setup

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import sys
import os

In [25]:
os. getcwd() 

'G:\\My Drive\\PhD Program\\4_Spring 2022\\Machine Learning for Cities\\machine-learning-final\\code'

In [5]:
root= r"G:\\My Drive\\PhD Program\\4_Spring 2022\\Machine Learning for Cities\\machine-learning-final"
inp= os.path.join(root, "data", "3_clean")

In [42]:
#from pydrive.auth import GoogleAuth

#gauth = GoogleAuth()
#gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

In [26]:
filepath = "G:/.shortcut-targets-by-id/1hgVHDpl_iZ7es6sKW0QRlC9X6SF58wk7/Machine Learning 4 Cities Final Project/Data/3_clean_data"

In [27]:

csv_path="G:/.shortcut-targets-by-id/1hgVHDpl_iZ7es6sKW0QRlC9X6SF58wk7/Machine Learning 4 Cities Final Project/Data/3_clean_data/analysis_bbl.csv"
df = pd.read_csv(csv_path, low_memory=False).drop("Unnamed: 0", axis = 1)


In [28]:
print(df.columns)

Index(['borough', 'bbl', 'cd', 'tract2010', 'zipcode', 'address', 'bldgclass',
       'landuse', 'ownertype', 'ownername',
       ...
       'pctnhasian', 'pctnhhawainpi', 'pctnhother', 'pctnhtwoplus',
       'pctownerocch', 'pctrentocch', 'crowding', 'crowdingrenter',
       'medianhhincome_imp', 'geofips_recode'],
      dtype='object', length=157)


In [29]:
df= pd.get_dummies(df, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [30]:
df.head()

,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,lotarea,...,pctrentocch,crowding,crowdingrenter,medianhhincome_imp,geofips_recode,borough_BK,borough_BX,borough_MN,borough_QN,borough_SI
0,1.012090e+09,107.0,181.0,10025.0,7 WEST 95 STREET,C2,2.0,NaN,ROBERT & ANNE RABBINO,2013.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
1,1.012090e+09,107.0,181.0,10025.0,5 WEST 95 STREET,C5,2.0,NaN,"5 WEST 95TH STREET,",2517.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
2,1.012090e+09,107.0,181.0,10025.0,12 WEST 96 STREET,D4,3.0,NaN,12 W 96TH STREET CORP,10570.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
3,1.012090e+09,107.0,181.0,10025.0,44 WEST 96 STREET,D4,3.0,NaN,44 OWNERS CORP,3926.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0
4,1.012090e+09,107.0,181.0,10025.0,46 WEST 96 STREET,D4,3.0,NaN,46 W 96TH ST OWNERS CORP,5035.0,...,67.120075,2.532833,3.773585,0.0,1018100.0,0,0,1,0,0


In [31]:
len(df. index) 

765085

In [32]:
#print(df.nunique())
print(df['vacate_ind'].value_counts())

0.0    762424
1.0      2604
Name: vacate_ind, dtype: int64


In [46]:
# define predictors
df.columns
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi"]
predictors_acs= df.columns[df.columns.get_loc("totpop"):df.columns.get_loc("crowdingrenter") + 1].tolist()
predictors_full= predictors + predictors_acs
predictors_full

['lotarea',
 'bldgarea',
 'numbldgs',
 'numfloors',
 'unitsres',
 'unitstotal',
 'assessland',
 'assesstot',
 'exempttot',
 'yearbuilt',
 'holc_AB',
 'holc_CD',
 'holc_D',
 'tot_hpd_vio',
 'hpd_fire_vio',
 'hpd_b_c_vio',
 'viol_count_per_unit',
 'hpd_fire_vio_per_unit',
 'hpd_b_c_vio_per_unit',
 'AL',
 'NB',
 'average_energy_usage',
 'average_energy_usage_mi',
 'totpop',
 'popdens',
 'areasqmile',
 'males',
 'females',
 'pop18under',
 'pop18_34',
 'pop35_64',
 'pop65plus',
 'pophispanic',
 'popnhwhite',
 'popnhblack',
 'popnhaian',
 'popnhasian',
 'popnhhawainpi',
 'popnhother',
 'popnhtwoplus',
 'housing',
 'housingoccupied',
 'renterhousing',
 'oc_halfless',
 'oc_half_1',
 'oc_1_onehalf',
 'oc_onehalf_2',
 'oc_2plus',
 'roc_halfless',
 'roc_half_1',
 'roc_1_onehalf',
 'roc_onehalf_2',
 'roc_2plus',
 'medianyearbuilt',
 'mediangrossrent',
 'medianhhincome',
 'gini',
 'year',
 'pct18under',
 'pct18_34',
 'pct35_64',
 'pct65plus',
 'pcthispanic',
 'pctnhwhite',
 'pctnhblack',
 'pctnhaia

In [33]:
X = df.loc[:, ["bldgclass", "landuse", "lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
              "assesstot", "exempttot", "yearbuilt", 'yearalter1', 'yearalter2', "holc_AB", "holc_CD", "holc_D", "viol_count", "AL", "NB",
              "average_energy_usage", "average_energy_usage_mi"]+borocols].fillna(0)

y = df.loc[:, "vacate_ind"]

In [49]:
predictors_fimp=["hpd_b_c_vio", "bldgarea", "tot_hpd_vio", "unitstotal", "pop65plus", "popdens", "gini", 
                 "holc_AB", "holc_CD", "holc_D" ] 
predictors_fimp

['hpd_b_c_vio',
 'bldgarea',
 'tot_hpd_vio',
 'unitstotal',
 'pop65plus',
 'popdens',
 'gini',
 'holc_AB',
 'holc_CD',
 'holc_D']

Using the Naive Bayes Classifier to predict whether a building had a fire vacate order.

In [54]:
#data1_X = data1.iloc[:,1:] # tip, distance, speed, and number of passengers
#data1_y = data1.iloc[:,0] # binary output: 1 if in Manhattan, 0 if outside

X = df.loc[:, predictors_fimp].fillna(0)
y = df.loc[:, "vacate_ind"].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [55]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
def trainNaiveBayesDiscrete(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
        nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
    return nbc

def testNaiveBayesDiscrete(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            if thevalue not in nbc[j+'_1']:
                y_pred[i] = y_pred[i]*1E-3
            if thevalue not in nbc[j+'_0']:
                y_pred[i] = y_pred[i]*1E3
            if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
                y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [56]:
naive_bayes_classifier = trainNaiveBayesDiscrete(X_train,y_train)
for i,j in naive_bayes_classifier.items():
    print(i)
    print(j)
    print()
y_pred_train = testNaiveBayesDiscrete(X_train,naive_bayes_classifier)
y_pred_test = testNaiveBayesDiscrete(X_test,naive_bayes_classifier)

prior
0.003440145134390472

hpd_b_c_vio_1
4.0       0.026342
7.0       0.020263
8.0       0.017730
11.0      0.017224
9.0       0.017224
            ...   
384.0     0.000507
259.0     0.000507
460.0     0.000507
1214.0    0.000507
370.0     0.000507
Name: hpd_b_c_vio, Length: 554, dtype: float64

hpd_b_c_vio_0
0.0       0.875304
1.0       0.020095
2.0       0.012222
3.0       0.009209
4.0       0.007159
            ...   
847.0     0.000002
476.0     0.000002
1828.0    0.000002
542.0     0.000002
591.0     0.000002
Name: hpd_b_c_vio, Length: 832, dtype: float64

bldgarea_1
4875.0     0.005572
3600.0     0.003546
3000.0     0.003546
3198.0     0.003546
2400.0     0.003040
             ...   
63460.0    0.000507
19270.0    0.000507
46505.0    0.000507
99344.0    0.000507
6308.0     0.000507
Name: bldgarea, Length: 1691, dtype: float64

bldgarea_0
2400.0     0.008249
1600.0     0.007696
2000.0     0.007214
1440.0     0.006950
1800.0     0.006944
             ...   
59352.0    0.000002
25

In [57]:

# measure accuracy for the binary prediction task
print('In sample prediction accuracy:',1.0*sum((y_pred_train>0.5)==y_train)/len(y_train))
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))

# measure accuracy of the predicted probabilities
print('Log-likelihood (train):',sum(np.log(y_pred_train*y_train+(1-y_pred_train)*(1-y_train))))
print('Log-likelihood (test):',sum(np.log(y_pred_test*y_test+(1-y_pred_test)*(1-y_test))))

In sample prediction accuracy: 0.9957041754020909
Out of sample prediction accuracy: 0.9904272449705132
Log-likelihood (train): -5650.63557772144
Log-likelihood (test): -8816.846070334337


In [ ]:
#Calculate precision and recall





### Try with the full set of variables

In [62]:
X = df.loc[:, predictors].fillna(0)
X.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio,viol_count_per_unit,hpd_fire_vio_per_unit,hpd_b_c_vio_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,11.0,0.0,8.0,1.100000,0.0,0.8,0.0,0.0,0.0,1.0
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,56.0,0.0
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,1.0,0.0,0.0,0.040000,0.0,0.0,0.0,0.0,0.0,1.0
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,4.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0


In [63]:
y = df.loc[:, "vacate_ind"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [64]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
def trainNaiveBayesDiscrete(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
        nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
    return nbc

def testNaiveBayesDiscrete(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            if thevalue not in nbc[j+'_1']:
                y_pred[i] = y_pred[i]*1E-3
            if thevalue not in nbc[j+'_0']:
                y_pred[i] = y_pred[i]*1E3
            if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
                y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [65]:
# measure accuracy for the binary prediction task
print('In sample prediction accuracy:',1.0*sum((y_pred_train>0.5)==y_train)/len(y_train))
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))

# measure accuracy of the predicted probabilities
print('Log-likelihood (train):',sum(np.log(y_pred_train*y_train+(1-y_pred_train)*(1-y_train))))
print('Log-likelihood (test):',sum(np.log(y_pred_test*y_test+(1-y_pred_test)*(1-y_test))))

In sample prediction accuracy: 0.9956292380967319
Out of sample prediction accuracy: 0.9903540507758585
Log-likelihood (train): nan
Log-likelihood (test): nan
